In [ ]:
# EX3: Find a dataset that is very large, so large that you can't see a gap between train and val loss. 
#      Pretrain the transformer on this data, then initialize with that model and finetune it on tiny 
#      shakespeare with a smaller number of steps and lower learning rate. Can you obtain a lower 
#      validation loss by the use of pretraining?